<a href="https://colab.research.google.com/github/AnshumanJain101/RecommendationCollaborative/blob/main/CollabFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
import sys
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from scipy import sparse

In [11]:
!pip install scikit-surprise

In [12]:
from __future__ import (absolute_import, division, print_function,             
                        unicode_literals)                                      
import pickle
import os
from surprise import KNNBasic
from surprise import Dataset                                                     
from surprise import Reader                                                      
from surprise.model_selection import PredefinedKFold
from surprise import dump
from surprise.accuracy import rmse
from surprise.accuracy import mae
from surprise import SVD
import os

In [13]:
# df is dataframe
df = pd.read_csv('ratings.csv', ',')
print(df.head())

n_users = df.userId.unique().shape[0]
n_items = df.movieId.unique().shape[0]
print ('Total Users : '+ str(n_users))
print('Total Movie : ' + str(n_items))

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
Total Users : 610
Total Movie : 9724


In [14]:
# converting data frame to utility matrix .
df_utility = df.pivot(index = 'userId',columns='movieId',values = 'rating').fillna(0)
print(df_utility.head())

#creating numpy array from pandas dataframe cz it provides better computn

ratings = np.array(df_utility)
# ratings numpy utility matrix

print(ratings)

movieId  1       2       3       4       ...  193583  193585  193587  193609
userId                                   ...                                
1           4.0     0.0     4.0     0.0  ...     0.0     0.0     0.0     0.0
2           0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
3           0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
4           0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
5           4.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0

[5 rows x 9724 columns]
[[4.  0.  4.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [2.5 2.  2.  ... 0.  0.  0. ]
 [3.  0.  0.  ... 0.  0.  0. ]
 [5.  0.  0.  ... 0.  0.  0. ]]


In [15]:
def pearson_sim(mat):
  sim_matrix = np.zeros((mat.shape[0],mat.shape[1]))
  #print(mat.shape[0])
  mean = np.mean(mat,axis=1)
  #print(mean)
  for user in range(mat.shape[0]):
    nonzeroarr = mat[user,:].nonzero()[0]
    #print(nonzeroarr)
    avg = np.sum(mat[user])/len(nonzeroarr)
    #print(avg)
    sim_matrix[user,nonzeroarr] = mat[user,nonzeroarr] - avg + 1e-9
	
  sim_matrix = (sim_matrix).dot((sim_matrix).T)
  #print(sim_matrix)
  norms = np.array([np.sqrt(np.diagonal(np.abs(sim_matrix)))])
  #print(norms)
	# norms is a square root array of magnitude of each user (diagonal contains magntitude of rows)
  #print(sim_matrix / norms / norms.T)
  return (sim_matrix / norms / norms.T)

In [16]:
  # nonzeroarr = ratings[2,:].nonzero()[0]
  # print(nonzeroarr)
  # nonzerolen = len(nonzeroarr)
  # print(nonzerolen)
  # test_rating_indices = np.random.choice(nonzeroarr,size=int(nonzerolen*0.10),replace=False)
  # print(test_rating_indices) 
  # the cocept of splitting is ki 10 maan lo ratings maine train mai 0 kr di aur unke corresponding test mai daal di  
  def train_test_split(ratings,fractionTest):
    test = np.zeros(ratings.shape)
        
    train = ratings.copy()
    for user in range(ratings.shape[0]):
      nonzeroarr = ratings[user,:].nonzero()[0]
      # print(nonzeroarr[0])
      nonzerolen = len(nonzeroarr)
      # print(nonzerolen)
      test_rating_indices = np.random.choice(nonzeroarr,size=int(nonzerolen*fractionTest),replace=False)
      #training_rating_ind = np.random.choice(nonzeroarr,size=int(nonzerolen*(1-fractionTest)),replace=False)
      train[user,test_rating_indices]=0
      test[user,test_rating_indices] = ratings[user,test_rating_indices]

    assert(np.all((train*test)==0))
    return train,test

In [17]:
def predict(ratings,similarity):
	# Summation sim(u,u')*r(u',i) / Summation of |sim(u,u')|
  #den2=np.array(np.abs(similarity)).T
  #print(den2)
  den = np.array(np.abs(similarity).sum(axis=1)).T
  #print(den)
  den = den.reshape((den.shape[0],1))
  #print(similarity.dot(ratings))
  return similarity.dot(ratings) /den

In [18]:
def predict_better(ratings,similarity):
	# r(ui) = ravg(u) + sum((sim(u,u')*(ru'i - ravgu'))/sum(sim(u,u'))
  user_bias = ratings.mean(axis=1)
  pred2= user_bias[:, np.newaxis]
  #print(pred2)
  ratings = (ratings - user_bias[:, np.newaxis]).copy()
  pred = similarity.dot(ratings) / np.array([np.abs(similarity).sum(axis=1)]).T
  pred += user_bias[:, np.newaxis]
  return pred

In [19]:
def get_rmse(pred, actual):
    # Ignore zero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [20]:
def get_mae(pred, actual):
    # Ignore zero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_absolute_error(pred, actual)

In [21]:
#reason for taking k as 50 ki maine try kiya and this was giving best result

def predict_topk(ratings,similarity,k=50,bias=0):

    # make new similarity matrix having only top k similar users
    pred = np.zeros(ratings.shape)
    new_sim = np.zeros(similarity.shape)
    
    for i in range(similarity.shape[0]):
        x = tuple(np.sort(np.argsort(similarity[:,i])[:-k-1:-1]))
        new_sim[i][[x]] = similarity[i][[x]]
     
    return predict_better(ratings,new_sim)

In [24]:
split_ratio = [0.30]
MAE_pb = []
for split in split_ratio:
    print("****************************************************************\n\n\n\n")
    print("split = ")
    print(split)
    #print("\n\n\n")
    train,test = train_test_split(ratings,split) 
    #print('Data splitted into Train,Test')
    #print(train)
    sim_matrix = pearson_sim(train)
    #sim_matrix2= train(method='pearson')
    #print(sim_matrix)
    #print(test)
    algo = KNNBasic(sim_options = {'name':'pearson','user_based': True})

    Dataset.load_builtin('ml-100k')

    train_file = os.path.expanduser('~') + '/.surprise_data/ml-100k/ml-100k/u1.base'
    test_file = os.path.expanduser('~') + '/.surprise_data/ml-100k/ml-100k/u1.test'
    data = Dataset.load_from_folds([(train_file, test_file)], Reader('ml-100k'))

    pkf = PredefinedKFold()
    print(pkf)
    print("\n\n")


    for trainset, testset in pkf.split(data):
        algo.fit(trainset)                             
        predictions = algo.test(testset)
        print(rmse(predictions))
        print("\n")
        print(mae(predictions))
                                                                                
        #dump.dump('./dump_file', predictions, algo)


    user_prediction_pearson_bias = predict(train,sim_matrix)
    #print(user_prediction_pearson_bias)
    #print(test)
    #print ('User-based CF MSE:(Pearson)' + str(get_rmse(user_prediction_pearson_bias, test)))
    user_prediction_pearson_nobias = predict_nobias(train,sim_matrix)
    #print ('User-based CF MSE:(Pearson)' + str(get_rmse(user_prediction_pearson_nobias, test)))
    #print ('User-based CF MSE:(Pearson)' + str(get_mae(user_prediction_pearson_nobias, test)))
    #mae_pb=compute_MAE(user_prediction_pearson_nobias, test)
    #MAE_pb.append(mae_pb[0])
    #print ('User-based CF MAE:(Pearson)' +str((compute_MAE(user_prediction_pearson_nobias, test))[0]))

    user_prediction_pearson_nobias_topk = predict_topk(train,sim_matrix)
    #print ('User-based CF MSE:(Pearson,TOP-K) without bias ' + str(get_rmse(user_prediction_pearson_nobias_topk, test)))
    #print ('User-based CF MAE:(Pearson,TOP-K) without bias ' + str(get_mae(user_prediction_pearson_nobias_topk, test)))


****************************************************************




split = 
0.3
Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k



Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.0205
1.0205088226677514


MAE:  0.8114
0.8113987415916873


NameError: ignored